<a href="https://colab.research.google.com/github/thomson008/pdiot-cw3/blob/main/Data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from scipy import stats
import tensorflow as tf
from keras import Sequential
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/pdiot-cw3/data'

file_list = []
sensor_position = 'Chest_Right'

for root, dirs, files in os.walk(path):
    for file in files:
        if ('csv' in file and sensor_position in file):
            file_list.append(os.path.join(root, file))
            
dataframes = []

for file in file_list:
    with open(file) as f:
        info = [next(f).rstrip().split(': ')[1] for x in range(5)]
        user_id = info[4]
        activity = info[2]
        
    file_data = pd.read_csv(file, header=5)
    if (file_data.size):
        dataframes.append(file_data)
        file_data.insert(0, 'user_id', user_id)
        file_data.insert(1, 'activity', activity)
    

sensor_data = pd.concat(dataframes, ignore_index=True).drop('seq', axis=1)

In [ ]:
scale_columns = ['accel_x', 'accel_y', 'accel_z']

def split_data(leave_subject):
    subjects = sensor_data.user_id.unique()
    train_subjects, test_subjects = subjects[: int(0.98*len(subjects))], subjects[int(0.98*len(subjects)):]

    df_train, df_test = sensor_data[sensor_data.user_id != leave_subject], sensor_data[sensor_data.user_id == leave_subject]

    scaler = RobustScaler()

    scaler = scaler.fit(df_train[scale_columns])

    df_train.loc[:, scale_columns] = scaler.transform(
      df_train.loc[:, scale_columns].to_numpy()
    )

    df_test.loc[:, scale_columns] = scaler.transform(
      df_test.loc[:, scale_columns].to_numpy()
    )

    return df_train, df_test

In [ ]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

TIME_STEPS = 36
STEP = 10

def create_whole_dataset(time_steps, step):
    df_train, df_test = split_data('s1758009')
    X_train, y_train = create_dataset(
        df_train[scale_columns],
        df_train.activity,
        time_steps,
        step
    )

    X_test, y_test = create_dataset(
        df_test[scale_columns],
        df_test.activity,
        time_steps,
        step
    )

    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    enc = enc.fit(y_train)

    y_train = enc.transform(y_train)
    y_test = enc.transform(y_test)

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = create_whole_dataset(TIME_STEPS, STEP)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [ ]:
def create_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 1, 10, 32
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
	model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
	model.add(Dropout(0.5))
	model.add(MaxPooling1D(pool_size=2))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model

model = create_model(X_train, y_train, X_test, y_test)

# evaluate model
_, accuracy = model.evaluate(X_test, y_test, batch_size=32, verbose=0)

Epoch 1/10
212/212 [==============================] - 2s 8ms/step - loss: 1.2428 - accuracy: 0.5717
Epoch 2/10
212/212 [==============================] - 2s 8ms/step - loss: 0.7507 - accuracy: 0.7284
Epoch 3/10
212/212 [==============================] - 2s 8ms/step - loss: 0.6210 - accuracy: 0.7723
Epoch 4/10
212/212 [==============================] - 2s 8ms/step - loss: 0.5081 - accuracy: 0.8250
Epoch 5/10
212/212 [==============================] - 2s 8ms/step - loss: 0.4483 - accuracy: 0.8409
Epoch 6/10
212/212 [==============================] - 2s 8ms/step - loss: 0.3992 - accuracy: 0.8563
Epoch 7/10
212/212 [==============================] - 2s 8ms/step - loss: 0.3707 - accuracy: 0.8653
Epoch 8/10
212/212 [==============================] - 2s 8ms/step - loss: 0.3392 - accuracy: 0.8782
Epoch 9/10
212/212 [==============================] - 2s 8ms/step - loss: 0.3118 - accuracy: 0.8857
Epoch 10/10
212/212 [==============================] - 2s 8ms/step - loss: 0.3095 - accuracy: 0.8891

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmp_cf5987n/assets


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 34, 64)            640       
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 32, 64)            12352     
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 64)            0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 16, 64)            0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               102500    
_________________________________________________________________
dense_7 (Dense)              (None, 15)               

In [ ]:
model.input_shape

(None, 36, 3)